# Imports

In [1]:
# Data Management
import pandas as pd
import numpy as np
import re
from pandas_datareader.data import DataReader
from ta import add_all_ta_features

# Statistics
from statsmodels.tsa.stattools import adfuller

# Data Preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, MinMaxScaler

# Supervised Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Reporting
import matplotlib.pyplot as plt

# Data Ingestion

In [2]:
df = pd.read_csv("../sydney-house-prices.csv")

In [3]:
print(f"Amount of rows: {df.shape[0]}")
print(f"Amount of columns: {df.shape[1]}")
df.head()

Amount of rows: 199504
Amount of columns: 9


,Date,Id,suburb,postalCode,sellPrice,bed,bath,car,propType
0,2019-06-19,1,Avalon Beach,2107,1210000,4.0,2,2.0,house
1,2019-06-13,2,Avalon Beach,2107,2250000,4.0,3,4.0,house
2,2019-06-07,3,Whale Beach,2107,2920000,3.0,3,2.0,house
3,2019-05-28,4,Avalon Beach,2107,1530000,3.0,1,2.0,house
4,2019-05-22,5,Whale Beach,2107,8000000,5.0,4,4.0,house


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199504 entries, 0 to 199503
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Date        199504 non-null  object 
 1   Id          199504 non-null  int64  
 2   suburb      199504 non-null  object 
 3   postalCode  199504 non-null  int64  
 4   sellPrice   199504 non-null  int64  
 5   bed         199350 non-null  float64
 6   bath        199504 non-null  int64  
 7   car         181353 non-null  float64
 8   propType    199504 non-null  object 
dtypes: float64(2), int64(4), object(3)
memory usage: 13.7+ MB


In [5]:
df.describe()

,Id,postalCode,sellPrice,bed,bath,car
count,199504.00000,199504.000000,1.995040e+05,199350.000000,199504.000000,181353.000000
mean,99752.50000,2196.379155,1.269776e+06,3.516479,1.890669,1.936224
std,57591.98839,193.053467,6.948239e+06,1.066555,0.926001,1.060237
min,1.00000,2000.000000,1.000000e+00,1.000000,1.000000,1.000000
25%,49876.75000,2082.000000,7.200000e+05,3.000000,1.000000,1.000000
50%,99752.50000,2144.000000,9.850000e+05,3.000000,2.000000,2.000000
75%,149628.25000,2211.000000,1.475000e+06,4.000000,2.000000,2.000000
max,199504.00000,4878.000000,2.147484e+09,99.000000,99.000000,41.000000


In [6]:
# Function to unify column names
def unify_column_names(name: str) -> str:
    return re.sub(r'([a-z])([A-Z])', r'\1_\2', name).lower()

In [7]:
df.columns = df.columns.map(lambda x: unify_column_names(x))

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199504 entries, 0 to 199503
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   date         199504 non-null  object 
 1   id           199504 non-null  int64  
 2   suburb       199504 non-null  object 
 3   postal_code  199504 non-null  int64  
 4   sell_price   199504 non-null  int64  
 5   bed          199350 non-null  float64
 6   bath         199504 non-null  int64  
 7   car          181353 non-null  float64
 8   prop_type    199504 non-null  object 
dtypes: float64(2), int64(4), object(3)
memory usage: 13.7+ MB


# Feature Engineering

### Handle Non-numerical Data

In [9]:
# Count unique items for 'suburb'
suburb_unique_text = df['suburb'].nunique()
print(f'Number of unique suburbs: {suburb_unique_text}')
print("Will require label encoding.")

Number of unique suburbs: 685
Will require label encoding.


In [10]:
# Perform label encoding and assign a number to each 'suburb'
label_encoder = LabelEncoder()
encoded_suburbs = label_encoder.fit_transform(df['suburb'])
df['suburbs_encoded'] = encoded_suburbs
df.head(40)

,date,id,suburb,postal_code,sell_price,bed,bath,car,prop_type,suburbs_encoded
0,2019-06-19,1,Avalon Beach,2107,1210000,4.0,2,2.0,house,22
1,2019-06-13,2,Avalon Beach,2107,2250000,4.0,3,4.0,house,22
2,2019-06-07,3,Whale Beach,2107,2920000,3.0,3,2.0,house,654
3,2019-05-28,4,Avalon Beach,2107,1530000,3.0,1,2.0,house,22
4,2019-05-22,5,Whale Beach,2107,8000000,5.0,4,4.0,house,654
5,2019-05-10,6,Bilgola Plateau,2107,1610000,4.0,2,2.0,house,67
6,2019-04-13,7,Avalon Beach,2107,1510000,3.0,1,2.0,house,22
7,2019-04-11,8,Avalon Beach,2107,1500000,5.0,4,4.0,house,22
8,2019-04-11,9,Avalon Beach,2107,1400000,3.0,1,2.0,house,22
9,2019-04-09,10,Avalon Beach,2107,1210000,3.0,2,2.0,house,22


In [11]:
# Count unique items for 'prop_type'
prop_type_unique_text = df['prop_type'].nunique()
print(f'Number of unique property types: {prop_type_unique_text}')
print("Will require one hot encoding.")

Number of unique property types: 8
Will require one hot encoding.


In [12]:
# Perform one hot encoding and assign a number to each 'prop_type'
one_hot_encoder = OneHotEncoder(sparse_output=False) # Use sparse=False to get a dense array (not sparse matrix)

# Reshape the 'prop_type' column to be a 2D array
encoded_property_types = one_hot_encoder.fit_transform(df['prop_type'].values.reshape(-1, 1))

# Convert the result to a DataFrame from a Series
encoded_property_types_df = pd.DataFrame(encoded_property_types, columns=one_hot_encoder.categories_[0])

# Append the encoded columns to the original dataframe
df = pd.concat([df, encoded_property_types_df], axis=1)
df.head(40)

,date,id,suburb,postal_code,sell_price,bed,bath,car,prop_type,suburbs_encoded,acreage,duplex/semi-detached,house,other,terrace,townhouse,villa,warehouse
0,2019-06-19,1,Avalon Beach,2107,1210000,4.0,2,2.0,house,22,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2019-06-13,2,Avalon Beach,2107,2250000,4.0,3,4.0,house,22,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2019-06-07,3,Whale Beach,2107,2920000,3.0,3,2.0,house,654,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,2019-05-28,4,Avalon Beach,2107,1530000,3.0,1,2.0,house,22,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,2019-05-22,5,Whale Beach,2107,8000000,5.0,4,4.0,house,654,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,2019-05-10,6,Bilgola Plateau,2107,1610000,4.0,2,2.0,house,67,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,2019-04-13,7,Avalon Beach,2107,1510000,3.0,1,2.0,house,22,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,2019-04-11,8,Avalon Beach,2107,1500000,5.0,4,4.0,house,22,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8,2019-04-11,9,Avalon Beach,2107,1400000,3.0,1,2.0,house,22,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9,2019-04-09,10,Avalon Beach,2107,1210000,3.0,2,2.0,house,22,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


### Set Target

In [13]:
# Set target
df['target'] = df['sell_price']
df.head(20)

,date,id,suburb,postal_code,sell_price,bed,bath,car,prop_type,suburbs_encoded,acreage,duplex/semi-detached,house,other,terrace,townhouse,villa,warehouse,target
0,2019-06-19,1,Avalon Beach,2107,1210000,4.0,2,2.0,house,22,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1210000
1,2019-06-13,2,Avalon Beach,2107,2250000,4.0,3,4.0,house,22,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2250000
2,2019-06-07,3,Whale Beach,2107,2920000,3.0,3,2.0,house,654,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2920000
3,2019-05-28,4,Avalon Beach,2107,1530000,3.0,1,2.0,house,22,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1530000
4,2019-05-22,5,Whale Beach,2107,8000000,5.0,4,4.0,house,654,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,8000000
5,2019-05-10,6,Bilgola Plateau,2107,1610000,4.0,2,2.0,house,67,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1610000
6,2019-04-13,7,Avalon Beach,2107,1510000,3.0,1,2.0,house,22,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1510000
7,2019-04-11,8,Avalon Beach,2107,1500000,5.0,4,4.0,house,22,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1500000
8,2019-04-11,9,Avalon Beach,2107,1400000,3.0,1,2.0,house,22,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1400000
9,2019-04-09,10,Avalon Beach,2107,1210000,3.0,2,2.0,house,22,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1210000


### Remove Redundant Columns / Features

In [14]:
# Remove features
df_features_removed = df.copy()
df_features_removed.drop(columns=['date', 'id', 'suburb', 'prop_type', 'sell_price'], inplace=True)
df_features_removed.head(20)

,postal_code,bed,bath,car,suburbs_encoded,acreage,duplex/semi-detached,house,other,terrace,townhouse,villa,warehouse,target
0,2107,4.0,2,2.0,22,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1210000
1,2107,4.0,3,4.0,22,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2250000
2,2107,3.0,3,2.0,654,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2920000
3,2107,3.0,1,2.0,22,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1530000
4,2107,5.0,4,4.0,654,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,8000000
5,2107,4.0,2,2.0,67,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1610000
6,2107,3.0,1,2.0,22,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1510000
7,2107,5.0,4,4.0,22,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1500000
8,2107,3.0,1,2.0,22,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1400000
9,2107,3.0,2,2.0,22,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1210000


### Check for Missing & Outlier Values

In [15]:
# Check for NaN, None, or NA values
print(f'Total number of missing values: {df_features_removed.isna().sum().sum()}')

Total number of missing values: 18305


In [16]:
# Check for Infinite Values
print(f'Total number of infinite values: {df_features_removed.isin([np.inf, -np.inf]).sum().sum()}')

Total number of infinite values: 0


In [17]:
# Handle missing values by filling them in with the column mean
df_drop = df_features_removed.fillna(df_features_removed.mean())

In [18]:
# Check again if there are missing values to make sure they were filled with the column mean
print(f'Total number of missing values: {df_drop.isna().sum().sum()}')

Total number of missing values: 0


### Feature Scaling - Min Max Scaling

In [19]:
df_scaled = df_drop.copy()
# Ensure the dataframe is cast to a float type before scaling
df_scaled = df_scaled.astype('float64')
min_max_scaler = MinMaxScaler()
df_scaled.iloc[:] = min_max_scaler.fit_transform(df_scaled)
df_scaled.head(20)

,postal_code,bed,bath,car,suburbs_encoded,acreage,duplex/semi-detached,house,other,terrace,townhouse,villa,warehouse,target
0,0.037179,0.030612,0.010204,0.025,0.032164,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000563
1,0.037179,0.030612,0.020408,0.075,0.032164,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.001048
2,0.037179,0.020408,0.020408,0.025,0.956140,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.001360
3,0.037179,0.020408,0.000000,0.025,0.032164,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000712
4,0.037179,0.040816,0.030612,0.075,0.956140,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.003725
5,0.037179,0.030612,0.010204,0.025,0.097953,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000750
6,0.037179,0.020408,0.000000,0.025,0.032164,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000703
7,0.037179,0.040816,0.030612,0.075,0.032164,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000698
8,0.037179,0.020408,0.000000,0.025,0.032164,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000652
9,0.037179,0.020408,0.010204,0.025,0.032164,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000563


### Train - Test - Split

In [20]:
# Define features (X) and target (y)
X = df_features_removed.drop(columns=['target'])
y = df_features_removed['target']

In [21]:
X

,postal_code,bed,bath,car,suburbs_encoded,acreage,duplex/semi-detached,house,other,terrace,townhouse,villa,warehouse
0,2107,4.0,2,2.0,22,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2107,4.0,3,4.0,22,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2107,3.0,3,2.0,654,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,2107,3.0,1,2.0,22,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,2107,5.0,4,4.0,654,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199499,2234,5.0,3,7.0,318,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
199500,2234,4.0,3,2.0,318,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
199501,2234,4.0,2,2.0,5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
199502,2234,3.0,2,2.0,318,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [22]:
y

0         1210000
1         2250000
2         2920000
3         1530000
4         8000000
           ...   
199499    1900000
199500     980000
199501     850000
199502     640000
199503    1611000
Name: target, Length: 199504, dtype: int64

In [23]:
# Train, Test, Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (159603, 13)
X_test shape: (39901, 13)
y_train shape: (159603,)
y_test shape: (39901,)


### Machine Learning

In [24]:
# Train regressor
regressor = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
regressor.fit(X_train, y_train)

RandomForestRegressor(max_depth=10, random_state=42)

In [25]:
# Make predictions on the test set
y_pred = regressor.predict(X_test)
print(f"Test Predictions: {y_pred[:5]}")
print(f"Test Actual: {y_test[:5].values}")

Test Predictions: [1395672.96415101  772493.83443856 1387526.0155658  1508002.81815282
 1203014.06617312]
Test Actual: [3150000  531000 1837500 2765000  940500]


In [26]:
# Check accuracy
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)
n_scores = cross_val_score(regressor, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score="raise")

In [27]:
# Report performance (mean and standard deviation of MAE)
mae = n_scores.mean()
mae_std = n_scores.std()
print(f"Mean Absolute Error: {abs(mae)}")
print(f"Mean Absolute Error Standard Deviation: {mae_std}")

Mean Absolute Error: 359885.5175854826
Mean Absolute Error Standard Deviation: 22234.42846766613


In [28]:
# Additional Evaluation: R^2 and RMSE on the test set
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"R^2 Score: {r2}")
print(f"RMSE: {rmse}")

R^2 Score: -0.0019295494407027824
RMSE: 11033588.523050632
